# **EXERCÍCIO RFV USANDO K-MEANS**
---
---

## **1.Introdução**

**RFV** significa recência, frequência, valor e é utilizado para segmentação de clientes baseado no comportamento de compras dos clientes e agrupa eles em clusters parecidos. Utilizando esse tipo de agrupamento podemos realizar ações de marketing e CRM melhores direcionadas, ajudando assim na personalização do conteúdo e até a retenção de clientes.

Para cada cliente é preciso calcular cada uma das componentes abaixo:

- Recência (R): Quantidade de dias desde a última compra.
- Frequência (F): Quantidade total de compras no período.
- Valor (V): Total de dinheiro gasto nas compras do período.


### **1.Importando as Biblioteca e Dados**

In [42]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Esta é uma base de compras de clientes de um intervalo de 1 ano.

In [43]:
df_compras = pd.read_csv('dados_input 1.csv',
                         infer_datetime_format=True,
                         parse_dates=['DiaCompra'])
df_compras.head(5)

<ipython-input-43-7b7b11d8e342>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_compras = pd.read_csv('dados_input 1.csv',


,ID_cliente,CodigoCompra,DiaCompra,ValorTotal
0,12747,537215,2020-12-05,358.56
1,12747,538537,2020-12-13,347.71
2,12747,541677,2021-01-20,303.04
3,12747,545321,2021-03-01,310.78
4,12747,551992,2021-05-05,442.96


## **2.Fazendo o Tratamento da Base**

### 2.1.Conferindo se há valores missings

In [44]:
df_compras.isna().sum()

,0
ID_cliente,0
CodigoCompra,0
DiaCompra,0
ValorTotal,0


### 2.2.Criando a coluna **RECÊNCIA**

In [45]:
#Definindo o último dia da compra com o dia atual/hoje
dia_atual = df_compras['DiaCompra'].max()
print(dia_atual)

2021-12-09 00:00:00


In [46]:
#Agrupando pelo o ID do cliente e conferindo o último dia de compra
df_recencia = df_compras.groupby(by='ID_cliente',
                                 as_index=False)['DiaCompra'].max()
df_recencia.columns = ['ID_cliente', 'DiaUltimaCompra']
df_recencia.head()

,ID_cliente,DiaUltimaCompra
0,12747,2021-12-07
1,12748,2021-12-09
2,12749,2021-12-06
3,12820,2021-12-06
4,12821,2021-05-09


In [47]:
#Calculando a recência
df_recencia['Recencia'] = df_recencia['DiaUltimaCompra'].apply(
    lambda x: (dia_atual - x).days)

#Retirando a coluna DiaUltimaCompra
df_recencia.drop('DiaUltimaCompra', axis=1, inplace=True)
df_recencia.head()

,ID_cliente,Recencia
0,12747,2
1,12748,0
2,12749,3
3,12820,3
4,12821,214


### 2.3.Criando a coluna **FREQUÊNCIA**

In [48]:
#Selecionando apenas as colunas de id e código da compra,
#Fazendo agrupamento pelo o ID e fazendo uma contagem
df_frequencia = df_compras[['ID_cliente', 'CodigoCompra'
                            ]].groupby('ID_cliente').count().reset_index()
df_frequencia.columns = ['ID_cliente', 'Frequencia']
df_frequencia.head()

,ID_cliente,Frequencia
0,12747,11
1,12748,178
2,12749,5
3,12820,4
4,12821,1


### 2.4.Criando a coluna **VALOR**

In [49]:
df_valor = df_compras[['ID_cliente', 'ValorTotal'
                       ]].groupby('ID_cliente').sum().reset_index()
df_valor.columns = ['ID_cliente', 'Valor']
df_valor.head()

,ID_cliente,Valor
0,12747,4196.01
1,12748,31533.04
2,12749,4090.88
3,12820,942.34
4,12821,92.72


### 2.5.Criando a Tabela RFV

In [50]:
df_RF = df_recencia.merge(df_frequencia, on='ID_cliente')
df_RFV = df_RF.merge(df_valor, on='ID_cliente')
df_RFV.set_index('ID_cliente', inplace=True)
df_RFV.head()

,Recencia,Frequencia,Valor
ID_cliente,,,
12747,2,11,4196.01
12748,0,178,31533.04
12749,3,5,4090.88
12820,3,4,942.34
12821,214,1,92.72


## **3.Utilizando o K-means**

### 3.1.Padronizando a Base

In [51]:
df_RFV_pad = pd.DataFrame(StandardScaler().fit_transform(df_RFV),
                          columns=df_RFV.columns,
                          index=df_RFV.index)
df_RFV_pad.head()

,Recencia,Frequencia,Valor
ID_cliente,,,
12747,-0.903303,1.029065,0.991646
12748,-0.923350,26.055140,10.839744
12749,-0.893280,0.129924,0.953773
12820,-0.893280,-0.019932,-0.180481
12821,1.221700,-0.469502,-0.486554


### 3.2.Clusterizando com 4 grupos

In [52]:
#Criando os Clusters
cluster = KMeans(n_clusters=4, random_state=10)
cluster.fit(df_RFV_pad)

KMeans(n_clusters=4, random_state=10)

In [53]:
#Adicionando ao DataFrame as categorias
df_RFV['Grupo'] = pd.Categorical(cluster.labels_)
df_RFV.head()

,Recencia,Frequencia,Valor,Grupo
ID_cliente,,,,
12747,2,11,4196.01,2
12748,0,178,31533.04,1
12749,3,5,4090.88,0
12820,3,4,942.34,0
12821,214,1,92.72,3


## **4.Descrevendo Os Grupos**

In [54]:
df_RFV.groupby('Grupo').agg([ 'mean'])

<ipython-input-54-c761be2864a5>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_RFV.groupby('Grupo').agg([ 'mean'])


,Recencia,Frequencia,Valor
,mean,mean,mean
Grupo,,,
0,43.579464,3.412609,1098.270541
1,5.866667,71.866667,30331.182000
2,16.163498,17.201521,7043.572776
3,245.807179,1.523077,425.774585


Conclusão sobre os grupos:
* O Grupo 1 é o grupo o quais estão os melhores clientes, pois possuem uma recência baixa, costumam comprar mais que os demais grupos e tem um valor absoluto maior.
* O Grupo 2 é o segundo melhor compararando esses mesmo fatores.
* O Grupo 0 é o terceiro
* O Grupo 3 é o pior em todos os quesitos.

### 4.1.Renomeando os Grupos

In [55]:
df_RFV['Grupo'] = df_RFV['Grupo'].replace({0: 'C', 1: 'A', 2: 'B', 3: 'D'})

<ipython-input-55-7340a5a79763>:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_RFV['Grupo'] = df_RFV['Grupo'].replace({0: 'C', 1: 'A', 2: 'B', 3: 'D'})


In [56]:
df_RFV.groupby('Grupo').agg([ 'mean']).sort_values(by=('Valor', 'mean'), ascending=False)

<ipython-input-56-8c30cf2d546a>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_RFV.groupby('Grupo').agg([ 'mean']).sort_values(by=('Valor', 'mean'), ascending=False)


,Recencia,Frequencia,Valor
,mean,mean,mean
Grupo,,,
A,5.866667,71.866667,30331.182000
B,16.163498,17.201521,7043.572776
C,43.579464,3.412609,1098.270541
D,245.807179,1.523077,425.774585
